In [ ]:
import React from 'react';
function Jumbotron(props) {
    return (
        <div className="jumbotron text-center">
            <h1>{props.title}</h1>
        </div>
    );
}

In [ ]:
import React from 'react';
function Container(props) {
    return (
        <div className="container">
            {props.children}
        </div>
    );
}

In [27]:
import React from 'react';
function Card(props) {
    return (
        <div className='card'>
            <div className='card-body '>
                {props.children}
            </div>
        </div>
    );
}

In [28]:
import React from 'react';
function Col4(props) {
    return (
        <div className='col-4-sm'>
            {props.children}
        </div>
    );
}

In [29]:
import React from 'react';
function Row(props) {
    return (
        <div className='row'>
            {props.children}
        </div>
    );
}

In [30]:
import React from 'react';
function Greet(props) {
    return (
        <div>
            Hello {props.who}
        </div>
    );
}

In [ ]:
import React from 'react';
//import { useState, useEffect } from 'react';
const useState = React.useState;
const useEffect = React.useEffect;

function DataReader(props) {
    const [data, setData] = useState(false);
    
    useEffect(() => {
        //https://developer.mozilla.org/en-US/docs/Web/API/Fetch_API/Using_Fetch
        fetch('/api/students/' + props.id)
            .then(response => response.json())
            .then(jsonData => setData(jsonData))
            .catch(error => setData('An error occured'));
        
    }, [props.id]);
    
    if (data === false) {
        return (
            <Card>
                Loading...
            </Card>
        );
    } else {
        return (
            <Card>
                {JSON.stringify(data)}
            </Card>
        )
    }
}

In [32]:
import React from 'react';
function App(props) {
    return ( 
        <Container>
            <Jumbotron title='Hello 22-5KB'></Jumbotron>
            <Row>
                <Col4>
                    <Card>
                        <Greet who='22-5KB'></Greet>
                    </Card>
                    <DataReader id='258'>
                    </DataReader>
                </Col4>
            </Row>
        </Container>
    );
}

In [33]:
import fs from 'fs';
const mainTemplate = `
<!DOCTYPE html>
<html lang="en"><head>
    <title>{{TITLE}}</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.16.0/umd/popper.min.js"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.5.2/js/bootstrap.min.js"></script>
    <script src="https://unpkg.com/react@17/umd/react.development.js"></script>
    <script src="https://unpkg.com/react-dom@17/umd/react-dom.development.js"></script>

    <!-- Don't use this in production: -->
    <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>

</head>
<body>
    <div class="container">
        <div id='root'>
        </div>
    </div>
    {{SCRIPTS}}
    <script type="text/babel">
      ReactDOM.render(
        <App />,
        document.getElementById('root')
      );
    </script>
</body>
</html>
`

function createHtml(title='Bootstrap Demo', extraScripts='') {
    const extraScriptsEnv = '<script type="text/babel">' + extraScripts + '</script>';
    const withTitle = mainTemplate.replace('{{TITLE}}', title);
    const withScripts = withTitle.replace('{{SCRIPTS}}', extraScriptsEnv);
    return withScripts;
}

function createJS(notebookFileName='04A_client.ipynb') {
    const result = [];
    try {  
        const data = fs.readFileSync(notebookFileName, 'utf8');
        const dataJSON = JSON.parse(data)
        //console.log(dataJSON);

        for(const cell of dataJSON.cells) {
            if (cell['cell_type'] === 'markdown') {
                continue;
            }
            const source = cell['source'];
            if (source.length == 0) {
                continue
            }
            if (source[0].startsWith('import React from')) {
                source.shift() //remove first element
                result.push('\n', ...source, '\n') //append source
            }
            
        }
    } catch(e) {
        console.log('Error:', e.stack);
    }
    return result.join('');
    
}

In [47]:
const studentTable = [
    {'id': 258, 'fullname': 'George Burden', 'city': 'Prague', 'lledu': false},
    {'id': 263, 'fullname': 'Julia Seven', 'city': 'Brno', 'lledu': false},
    {'id': 396, 'fullname': 'Anthony Previous', 'city': 'Prague', 'lledu': true},
]

class studentController {
    constructor(data) {
        this.data = data;
    }
    
    get(id=-1) {
        if (id === -1) {
            return this.data;
        } else {
            for(const item of this.data) {
                if (item['id'] === id) {
                    return item;
                }
            }
            return null;
        }
        
    }
}

In [50]:
import express from 'express';

const app = express();

const students = new studentController(studentTable);

app.get('/api/students/:id', (req, res) => {
    const idInt = parseInt(req.params.id);
    const studentItem = students.get(idInt);
    if (studentItem === null) {
        res.status(404).send('Nenalezeno');
    } else {
        res.json(studentItem);
    }
});

app.get('/api/students', (req, res) => {
    res.json(students.get());
});

app.get('/', (req, res) => {
    const html = createHtml('Demo', createJS());
    res.send(html);
});

const srv = app.listen(9990, () =>{console.log('Server is running')});

Server is running
258
258
258
258


In [49]:
const serverResponse = srv.close(()=>{console.log('Server stopped')});

Server stopped
